In [20]:
import pandas as pd
import re
from textblob import TextBlob

In [2]:
data = pd.read_csv('data/twitter-airline-sentiment/Tweets.csv')
print(data.shape)
data.head()

(14640, 15)


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,5.703060e+17,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2/24/2015 11:35,NaN,Eastern Time (US & Canada)
1,5.703010e+17,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2/24/2015 11:15,NaN,Pacific Time (US & Canada)
2,5.703010e+17,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2/24/2015 11:15,Lets Play,Central Time (US & Canada)
3,5.703010e+17,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2/24/2015 11:15,NaN,Pacific Time (US & Canada)
4,5.703010e+17,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2/24/2015 11:14,NaN,Pacific Time (US & Canada)


In [8]:
tweets = pd.DataFrame(data['text'])
tweets.head(10)

,text
0,@VirginAmerica What @dhepburn said.
1,@VirginAmerica plus you've added commercials t...
2,@VirginAmerica I didn't today... Must mean I n...
3,@VirginAmerica it's really aggressive to blast...
4,@VirginAmerica and it's a really big bad thing...
5,@VirginAmerica seriously would pay $30 a fligh...
6,"@VirginAmerica yes, nearly every time I fly VX..."
7,@VirginAmerica Really missed a prime opportuni...
8,"@virginamerica Well, I didn't…but NOW I DO! :-D"
9,"@VirginAmerica it was amazing, and arrived an ..."


In [9]:
tweets['At'] = tweets['text'].str.extract(r'^(@\S+)')
tweets.head(10)

,text,At
0,@VirginAmerica What @dhepburn said.,@VirginAmerica
1,@VirginAmerica plus you've added commercials t...,@VirginAmerica
2,@VirginAmerica I didn't today... Must mean I n...,@VirginAmerica
3,@VirginAmerica it's really aggressive to blast...,@VirginAmerica
4,@VirginAmerica and it's a really big bad thing...,@VirginAmerica
5,@VirginAmerica seriously would pay $30 a fligh...,@VirginAmerica
6,"@VirginAmerica yes, nearly every time I fly VX...",@VirginAmerica
7,@VirginAmerica Really missed a prime opportuni...,@VirginAmerica
8,"@virginamerica Well, I didn't…but NOW I DO! :-D",@virginamerica
9,"@VirginAmerica it was amazing, and arrived an ...",@VirginAmerica


In [13]:
# for removing handles
def remove_handles(tweet):
    return re.sub(r'@\S+', '', tweet)

In [14]:
tweets['text'] = tweets['text'].apply(remove_handles)

In [16]:
tweets.head(10)

,text,At
0,What said.,@VirginAmerica
1,plus you've added commercials to the experien...,@VirginAmerica
2,I didn't today... Must mean I need to take an...,@VirginAmerica
3,"it's really aggressive to blast obnoxious ""en...",@VirginAmerica
4,and it's a really big bad thing about it,@VirginAmerica
5,seriously would pay $30 a flight for seats th...,@VirginAmerica
6,"yes, nearly every time I fly VX this “ear wor...",@VirginAmerica
7,Really missed a prime opportunity for Men Wit...,@VirginAmerica
8,"Well, I didn't…but NOW I DO! :-D",@virginamerica
9,"it was amazing, and arrived an hour early. Yo...",@VirginAmerica


In [18]:
def get_sentiment(dataframe, column):
    '''
    PARAMETERS - 
        dataframe - pandas DataFrame object
        column - column in dataframe which contains the text to which sentiment is applied 
    
    RETURNS - 
        pandas dataframe, polarity Subjectivity for each text
    
    '''
    text_column = dataframe[column]
    textblob_sentiment = text_column.apply(TextBlob)
    
    sentiment_values = [ {'Polarity': v.sentiment.polarity,
                         'Subjectivity': v.sentiment.subjectivity}
                   for v in textblob_sentiment.values]
    return pd.DataFrame(sentiment_values)

In [21]:
sentiment_df = get_sentiment(tweets,'text')

In [22]:
sentiment_df.head()

,Polarity,Subjectivity
0,0.000000,0.000000
1,0.000000,0.000000
2,-0.390625,0.687500
3,0.006250,0.350000
4,-0.350000,0.383333


In [24]:
#concat the two dataframes
tweets = pd.concat([tweets, sentiment_df], axis=1)

In [25]:
tweets.head()

,text,At,Polarity,Subjectivity
0,What said.,@VirginAmerica,0.000000,0.000000
1,plus you've added commercials to the experien...,@VirginAmerica,0.000000,0.000000
2,I didn't today... Must mean I need to take an...,@VirginAmerica,-0.390625,0.687500
3,"it's really aggressive to blast obnoxious ""en...",@VirginAmerica,0.006250,0.350000
4,and it's a really big bad thing about it,@VirginAmerica,-0.350000,0.383333


In [32]:
postive_tweets = tweets[tweets['Polarity'] > 0.5]
negative_tweets = tweets[tweets['Polarity'] < -0.5]
neutral_tweets = tweets[(tweets['Polarity'] < 0.1) & (tweets['Polarity']> -0.1)]

## Positive tweets

In [33]:
print('Number of postive tweets - ',len(postive_tweets))
postive_tweets.head(10)

Number of postive tweets -  1047


,text,At,Polarity,Subjectivity
8,"Well, I didn't…but NOW I DO! :-D",@virginamerica,1.000000,1.000000
19,you know what would be amazingly awesome? BOS...,@VirginAmerica,0.600000,0.966667
22,I love the hipster innovation. You are a feel...,@VirginAmerica,0.600000,0.600000
34,this is great news! America could start flig...,@VirginAmerica,1.000000,0.750000
35,Nice RT Vibe with the moodlight from takeoff ...,NaN,0.600000,1.000000
36,Moodlighting is the only way to fly! Best exp...,@VirginAmerica,0.587500,0.712500
42,plz help me win my bid upgrade for my flight ...,@VirginAmerica,1.000000,0.400000
43,I have an unused ticket but moved to a new ci...,@VirginAmerica,0.578788,0.751515
45,I'm #elevategold for a good reason: you rock!!,@VirginAmerica,1.000000,0.600000
51,Julie Andrews all the way though was very ...,@VirginAmerica,1.000000,1.000000


## Negative tweets

In [34]:
print('Number of negative tweets - ',len(negative_tweets))
negative_tweets.head(10)

Number of negative tweets -  599


,text,At,Polarity,Subjectivity
33,"awaiting my return phone call, just would pre...",@VirginAmerica,-0.750000,1.000000
84,it was a disappointing experience which will ...,@VirginAmerica,-0.600000,0.700000
114,come back to #PHL already. We need you to tak...,@VirginAmerica,-0.533333,0.666667
131,us too! Terrible airline! Just gave us a hot...,@VirginAmerica,-0.750000,1.000000
181,too bad you say it takes 10 to 14 days via YO...,@VirginAmerica,-0.875000,0.666667
187,for all my flight stuff wrong and did nothin...,@VirginAmerica,-0.750000,0.950000
354,I am deeply disappointed that your birthday p...,@VirginAmerica,-0.625000,0.625000
367,on VX399 from JFK to LA - dirty plane - not u...,@VirginAmerica,-0.600000,0.800000
411,all crap channels which is why I pay to watch...,@VirginAmerica,-0.800000,0.800000
446,"Never had a bad experience before, but this o...",@VirginAmerica,-0.700000,0.666667


## Neutral tweets 


In [35]:
print('Number of neutral tweets are - ',len(neutral_tweets))
neutral_tweets.head(10)

Number of neutral tweets are -  6730


,text,At,Polarity,Subjectivity
0,What said.,@VirginAmerica,0.000000,0.000000
1,plus you've added commercials to the experien...,@VirginAmerica,0.000000,0.000000
3,"it's really aggressive to blast obnoxious ""en...",@VirginAmerica,0.006250,0.350000
10,did you know that suicide is the second leadi...,@VirginAmerica,0.000000,0.000000
15,SFO-PDX schedule is still MIA.,@VirginAmerica,0.000000,0.000000
17,I flew from NYC to SFO last week and couldn'...,@VirginAmerica,0.047619,0.290079
18,I ❤️ flying ☺️👍,NaN,0.000000,0.000000
23,will you be making BOS&gt;LAS non stop perman...,@VirginAmerica,0.000000,0.000000
25,status match program. I applied and it's bee...,@VirginAmerica,0.000000,0.000000
26,What happened 2 ur vegan food options?! At le...,@VirginAmerica,-0.075000,0.331250
